In [ ]:
import cv2
import time
import numpy as np

# Load the pre-trained MobileNet SSD model for vehicle detection
net = cv2.dnn.readNetFromCaffe('C:/Users/maryc/Documents/overspeedigncar/MobileNetSSD_deploy.prototxt.txt', 'C:/Users/maryc/Documents/overspeedigncar/MobileNetSSD_deploy.caffemodel')

# Set the video source (update the path accordingly)
video_path = 'C:/Users/maryc/Documents/overspeedigncar/2165-155327596_tiny.mp4'
cap = cv2.VideoCapture(video_path)

# Initialize variables for speed calculation
ref_width = 0.955  # Width of a US quarter in meters (approximate)
vehicle_dict = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame for faster processing
    frame_resized = cv2.resize(frame, (300, 300))
    blob = cv2.dnn.blobFromImage(frame_resized, 0.007843, (300, 300), 127.5)

    # Pass the blob through the network
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.6:  # Confidence threshold
            class_id = int(detections[0, 0, i, 1])
            if class_id == 7:  # Class ID for car
                box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
                x, y, w, h = box.astype(int)

                # Calculate speed for each vehicle
                vehicle_id = f"{x}_{y}"  # Unique identifier for each vehicle
                if vehicle_id not in vehicle_dict:
                    vehicle_dict[vehicle_id] = (time.time(), None)
                else:
                    start_time, _ = vehicle_dict[vehicle_id]
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    speed_mps = ref_width / elapsed_time
                    speed_kph = speed_mps * 3.6

                    # Check if overspeeding
                    if speed_kph > 60:
                        print(f"Vehicle {vehicle_id}: Speed: {speed_kph:.2f} km/h (Overspeeding)")
                    else:
                        print(f"Vehicle {vehicle_id}: Speed: {speed_kph:.2f} km/h")

                    # Update start time for next frame
                    vehicle_dict[vehicle_id] = (end_time, None)

                # Draw bounding box around the car
                cv2.rectangle(frame, (x, y), (w, h), (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Car Speed Detection', frame)
    if cv2.waitKey(1) == 27:  # Press 'Esc' to exit
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Vehicle 228_144: Speed: 68.72 km/h (Overspeeding)
Vehicle 193_179: Speed: 70.16 km/h (Overspeeding)
Vehicle 373_200: Speed: 127.31 km/h (Overspeeding)
Vehicle 226_182: Speed: 0.88 km/h
Vehicle 214_210: Speed: 0.29 km/h
Vehicle 350_207: Speed: 0.22 km/h
Vehicle 200_187: Speed: 0.25 km/h
Vehicle 219_173: Speed: 0.43 km/h
Vehicle 203_189: Speed: 0.47 km/h
Vehicle 360_199: Speed: 0.13 km/h
Vehicle 361_199: Speed: 2.36 km/h
Vehicle 219_173: Speed: 0.30 km/h
Vehicle 210_185: Speed: 81.09 km/h (Overspeeding)
Vehicle 209_190: Speed: 0.18 km/h
Vehicle 207_197: Speed: 0.27 km/h
Vehicle 192_214: Speed: 0.11 km/h
Vehicle 361_203: Speed: 0.10 km/h
Vehicle 233_172: Speed: 0.11 km/h
Vehicle 223_185: Speed: 0.11 km/h
Vehicle 216_194: Speed: 0.31 km/h
Vehicle 282_223: Speed: 91.93 km/h (Overspeeding)
